In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from datetime import timedelta

pd.options.display.max_columns = None

In [2]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

url3 = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url3,error_bad_lines = False)
df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})
df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']] = df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']].fillna(0)

In [3]:
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    """Checks whether the daily increments column adds up to the cumulative column for testing, positive, and negative results.
    
    If the columns are not equal, ID's the earliest date which the numbers are misaligned and fills the incremental results with the cumulative results.
    After review, the testing df is only missing the first daily increments.
    Still might want to add a check in case filling the earliest date is not enough to equate the two columns."""
    for state in states:
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])] 

In [4]:
state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [5]:
statepop_2019est = {
        'Alaska': 731545,
        'Alabama': 4903185,
        'Arkansas': 3017804,
        'American Samoa': 55312,
        'Arizona': 7278717,
        'California': 39512223,
        'Colorado': 5758736,
        'Connecticut': 3565287,
        'District of Columbia': 705749,
        'Delaware': 973764,
        'Florida': 21477737,
        'Georgia': 10617423,
        'Guam': 167294,
        'Hawaii': 1415872,
        'Iowa': 3155070,
        'Idaho': 1787065,
        'Illinois': 12671821,
        'Indiana': 6732219,
        'Kansas': 2913314,
        'Kentucky': 4467673,
        'Louisiana': 4648794,
        'Massachusetts': 6892503,
        'Maryland': 6045680,
        'Maine': 1344212,
        'Michigan': 9986857,
        'Minnesota': 5639632,
        'Missouri': 6137428,
        'Northern Mariana Islands': 57216,
        'Mississippi': 2976149,
        'Montana': 1068778,
        'North Carolina': 10488084,
        'North Dakota': 762062,
        'Nebraska': 1934408,
        'New Hampshire': 1359711,
        'New Jersey': 8882190,
        'New Mexico': 2096829,
        'Nevada': 3080156,
        'New York': 19453561,
        'Ohio': 11689100,
        'Oklahoma': 3956971,
        'Oregon': 4217737,
        'Pennsylvania': 12801989,
        'Puerto Rico': 2933408,
        'Rhode Island': 1059361,
        'South Carolina': 5148714,
        'South Dakota': 884659,
        'Tennessee': 6829174,
        'Texas': 28995881,
        'Utah': 3205958,
        'Virginia': 8535519,
        'Virgin Islands': 30030,
        'Vermont': 623989,
        'Washington': 7614893,
        'Wisconsin': 5822434,
        'West Virginia': 1792147,
        'Wyoming': 578759}

In [6]:
fill_func(df_testing['Province_State'].unique())

#Update state names
df_testing = df_testing.replace({'Province_State':state_abbrev})

#add state populations
df_testing['Population'] = df_testing['Province_State'].map(statepop_2019est)

df_testing['Date'] = pd.to_datetime(df_testing['Date'].astype(dtype = 'str'))

In [7]:
ca_testing = df_testing[['Date','Province_State','positive','negative','hospitalizedCurrently','inIcuCurrently', 'death', 'positiveIncrease','negativeIncrease','totalTestResults','totalTestResultsIncrease','deathIncrease']].loc[df_testing['Province_State'] =='California'].copy().reset_index(drop=True)
ca_testing['Province_State'] = 'California'
ca_testing = ca_testing.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_testing = df_testing[['Province_State','Date','Population','positiveIncrease', 'totalTestResultsIncrease','totalTestResults','positive','negative']].copy()

In [8]:
#Feature creation
us_testing['Tests per 1000 Capita'] = us_testing['totalTestResults']/us_testing['Population']*1000
us_testing['Total Positivity Rate'] = us_testing['positive']/us_testing['totalTestResults']*100

ranks = []
for date in us_testing['Date'].unique():
    for ranking in us_testing.copy().loc[us_testing['Date']==date,'Tests per 1000 Capita'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Tests per Capita Daily Ranking': ranks})
us_testing = pd.concat([us_testing, rankings], axis=1)

In [9]:
check = 0
for state in df_testing['Province_State'].unique():
    for col in range(0,len(to_fix)):
        cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
        actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
        comparison = cumsum.eq(actualsum)
        if comparison[comparison == False].count() > 0:
            print(state, to_fix[col], actual[col])
            check += 1
if check == 0:
    print('All States checked')

All States checked


In [18]:
ca_testing.to_csv(r'X:\AC\Documents\Datasets\CA_daily_pivot.csv')
us_testing.to_csv(r'X:\AC\Documents\Datasets\US_testing_daily.csv')

In [ ]:
us_testing.head()

In [20]:
df_testing.head()

,Date,Province_State,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,recovered,dataQualityGrade,lastUpdateEt,dateModified,checkTimeEt,death,hospitalized,dateChecked,totalTestsViral,positiveTestsViral,negativeTestsViral,positiveCasesViral,deathConfirmed,deathProbable,fips,positiveIncrease,negativeIncrease,total,totalTestResults,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade,Population
0,2020-07-30,Alaska,3559.0,221498.0,NaN,45.0,NaN,NaN,NaN,3.0,NaN,885.0,A,7/30/2020 00:00,2020-07-30T00:00:00Z,07/29 20:00,23.0,NaN,2020-07-30T00:00:00Z,225057.0,NaN,NaN,3559.0,23.0,NaN,2,117.0,1276.0,225057,225057,1393,225057,1,0,d66445af73028a02a5d6713580a4c470cc603322,0,0,0,0,0,NaN,731545
1,2020-07-30,Alabama,85762.0,594346.0,NaN,1595.0,10070.0,NaN,1129.0,NaN,605.0,35401.0,B,7/30/2020 11:00,2020-07-30T11:00:00Z,07/30 07:00,1565.0,10070.0,2020-07-30T11:00:00Z,677841.0,NaN,NaN,83495.0,1516.0,49.0,1,1980.0,9853.0,680108,680108,11833,680108,27,177,5776f9724a3452e48705a9b2c73f52d3e631229d,0,0,0,0,0,NaN,4903185
2,2020-07-30,Arkansas,41759.0,460958.0,NaN,508.0,2747.0,NaN,NaN,108.0,387.0,34737.0,A,7/30/2020 14:40,2020-07-30T14:40:00Z,07/30 10:40,442.0,2747.0,2020-07-30T14:40:00Z,502717.0,NaN,NaN,41759.0,NaN,NaN,5,791.0,6321.0,502717,502717,7112,502717,8,0,bd75bdc3753d8af0a8614e5c5b7091ff017bc1bf,0,0,0,0,0,NaN,3017804
3,2020-07-30,American Samoa,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,7/16/2020 00:00,2020-07-16T00:00:00Z,07/15 20:00,0.0,NaN,2020-07-16T00:00:00Z,NaN,NaN,NaN,0.0,NaN,NaN,60,0.0,0.0,1037,1037,0,1037,0,0,2e44db50a98f764facdbbe5256f4c72a0ec98104,0,0,0,0,0,NaN,55312
4,2020-07-30,Arizona,170798.0,733156.0,NaN,2348.0,11172.0,758.0,NaN,531.0,NaN,22358.0,A+,7/30/2020 00:00,2020-07-30T00:00:00Z,07/29 20:00,3626.0,11172.0,2020-07-30T00:00:00Z,913266.0,NaN,NaN,137710.0,2431.0,152.0,4,2525.0,0.0,903954,903954,2525,903954,172,1112,655170e3baafd8660693233c3daf5a99250093d9,0,0,0,0,0,NaN,7278717


# Data Exploration

In [10]:
most_recent = df_testing.loc[(df_testing['Province_State']=='California')]['Date'].max()
last14 = [most_recent - timedelta(days=x) for x in range(14)]
most_recent = df_testing.loc[(df_testing['Province_State']=='California') & (df_testing['Date']==most_recent)]
last14 = df_testing.loc[(df_testing['Province_State']=='California') & (df_testing['Date'].isin(last14))]

current_hosp = most_recent['hospitalizedCurrently'].iloc[0].astype(int)
perc_pos = (most_recent['positive']/most_recent['totalTestResults']).iloc[0] * 100
total_tests = most_recent['totalTestResults'].iloc[0]
population = statepop_2019est['California']
testsper1000 = total_tests/population*1000

last14tests = last14['totalTestResultsIncrease'].sum()
last14perc_pos = last14['positiveIncrease'].sum()/last14tests * 100

print('California Stats as of: ', pd.datetime.strftime(most_recent['Date'].max(), '%m/%d/%y'))
print('Total population in CA: %d' %population)
print('\n')
print('Total tests conducted: %d' %total_tests)
print('Total tests conducted per 1000 people in CA: %.2f' %testsper1000)
print('Percentage of total tests confirmed positive: %.2f' %perc_pos,'%')
print('Percentage of total tests confirmed negative: %.2f' %(100 - perc_pos),'%')
print(f'Currently hospitalized: {current_hosp}')
print('\n')
print('Last 14 days:')
print('Total tests conducted in last 14 days: %d' %last14tests)
print('Percentage of tests in last 14 days confirmed positive: %.2f' %last14perc_pos,'%')
print('Tests conducted in last 14 days as percentage of totals tests conducted: %d' %(last14tests/total_tests*100), '%')
print('Positives in last 14 days as percentage of total positives: %.2f' %(last14['positiveIncrease'].sum()/most_recent['positive']*100),'%')

California Stats as of:  08/03/20
Total population in CA: 39512223


Total tests conducted: 8184696
Total tests conducted per 1000 people in CA: 207.14
Percentage of total tests confirmed positive: 6.29 %
Percentage of total tests confirmed negative: 93.71 %
Currently hospitalized: 7629


Last 14 days:
Total tests conducted in last 14 days: 1770375
Percentage of tests in last 14 days confirmed positive: 6.97 %
Tests conducted in last 14 days as percentage of totals tests conducted: 21 %
Positives in last 14 days as percentage of total positives: 23.96 %


In [17]:
us_testing.sort_values(['Date','Total Positivity Rate'], ascending = [False,False])[0:24]

,Province_State,Date,Population,positiveIncrease,totalTestResultsIncrease,totalTestResults,positive,negative,Tests per 1000 Capita,Total Positivity Rate,Tests per Capita Daily Ranking
4,Arizona,2020-07-30,7278717,2525.0,2525,903954,170798.0,733156.0,124.191393,18.894546,42.0
10,Florida,2020-07-30,21477737,9956.0,52352,3579117,461379.0,3117738.0,166.643115,12.890861,19.0
28,Mississippi,2020-07-30,2976149,1775.0,6577,449881,57579.0,392302.0,151.162123,12.798718,27.0
44,South Carolina,2020-07-30,5148714,1726.0,10401,691391,87572.0,603819.0,134.284212,12.666060,35.0
1,Alabama,2020-07-30,4903185,1980.0,11833,680108,85762.0,594346.0,138.707391,12.610056,33.0
11,Georgia,2020-07-30,10617423,3963.0,28925,1541389,182286.0,1359103.0,145.175435,11.826087,32.0
47,Texas,2020-07-30,28995881,8800.0,70106,3609474,412107.0,3197367.0,124.482301,11.417370,41.0
15,Idaho,2020-07-30,1787065,457.0,2477,175519,19679.0,155840.0,98.216349,11.211892,49.0
36,Nevada,2020-07-30,3080156,1018.0,6260,463645,46824.0,416821.0,150.526467,10.099106,28.0
21,Massachusetts,2020-07-30,6892503,414.0,12334,1169152,117098.0,1052054.0,169.626622,10.015635,18.0


# Forecasting

In [28]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA

from datetime import datetime

In [34]:
ca_cases = us_testing.loc[us_testing['Province_State']=='California', ['Date','positive']]
type(ca_cases['Date'].iloc[0])
# ca_log = np.log(ca_cases)
# plt.plot(ca_cases)

pandas._libs.tslibs.timestamps.Timestamp

# Appendix

In [ ]:
us_conf_daily = df_usconf.copy()
us_dead_daily = df_usdead.copy()

us_conf_daily.iloc[:,11:] = us_conf_daily.iloc[:,11:].diff(axis=1).fillna(0).astype('int')
us_dead_daily.iloc[:,12:] = us_dead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

us_conf_daily = us_conf_daily.groupby(['Province_State'])[us_conf_daily.columns.to_list()[11:]].agg('sum').reset_index()
us_dead_daily = us_dead_daily.groupby(['Province_State'])[us_dead_daily.columns.to_list()[11:]].agg('sum').reset_index()

us_conf_daily = us_conf_daily.melt(id_vars = us_conf_daily.columns[:1], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['Province_State','Date']).reset_index(drop=True)
us_dead_daily = us_dead_daily.melt(id_vars = us_dead_daily.columns[:2], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_daily = us_conf_daily.merge(us_dead_daily[['Province_State','Population','Date']], on = ['Province_State','Date'], how = 'left', suffixes = (False, False))
us_daily = us_daily[['Province_State','Population','Date']]
us_daily['Date'] = pd.to_datetime(us_daily['Date'])
us_daily = us_daily.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

In [ ]:
#there will be discrepancies between the daily confirmed cases from JHU and from CDHP.
#Should probably use all CDHP metrics for CA analysis and only population from JHU
ca_daily = us_daily.loc[us_daily['Province_State']=='California'].copy().reset_index(drop=True)
ca_daily = ca_daily.merge(ca_testing[['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']], on = ['Province_State', 'Date'], how = 'left', suffixes = (False, False))

us_daily = df_testing[['Province_State','Date','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease','totalTestResults','positive','negative']]

In [ ]:
df_pop = pd.read_csv(r'X:\AC\Documents\Datasets\US Census and OMB Data\2019 State Population Estimates.csv', delimiter = ',', encoding = "ISO-8859-1")

In [ ]:
pop_check = df_pop.merge(us_testing[['Province_State','Population']], how='inner', left_on='State Name', right_on='Province_State')

In [ ]:
pop_check.drop_duplicates(inplace=True)
pop_check['delta'] = pop_check['2019 Total Population Estimate'] - pop_check['Population']
pop_check.loc[pop_check['delta']!=0]

In [ ]:
#apply date format and convert to string
.apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))

In [ ]:
columns = test_df.columns
test_df.iloc[7694,columns.get_loc('totalTestResultsIncrease')] = 1
a = np.cumsum(test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResultsIncrease'].values,dtype=int)
b = test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResults'].values
np.array_equal(a,b)

In [ ]:
#filling missing starting values. May want to do this as function for all states
df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positiveIncrease','negativeIncrease','totalTestResultsIncrease']] = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positive','negative','totalTestResults']].values.tolist()

In [ ]:
#Update function with secondary check
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    for state in states:
        print(state)
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            print(cumsum.iloc[0], actualsum.iloc[0], comparison.iloc[0])
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])]
                print('Fixed in first: ', state,to_fix[col],actual[col])
            else:
                print('Arrays are equal')
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() == 0:
                print('Confirmed arrays are fixed')
            else:
                print('Failed check: ', state,to_fix[col],actual[col] ,'\n')

In [ ]:
#['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']
#hospitalized currently missing values from american samoa, Hawaii, Kansas, northern marinas, virginia
#inincu currently missing values from 28 states/provinces
df_testing.loc[df_testing['Date']=='2020-07-21',['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']]

In [ ]:
cumsum = df_testing.loc[df_testing['Province_State']=='NV'].sort_values(by='Date')['totalTestResultsIncrease'].cumsum()
actualsum = df_testing.loc[df_testing['Province_State']=='NV',['Date','totalTestResults']].sort_values(by='Date')['totalTestResults']
check = cumsum.eq(actualsum)
check[check==False].sum()

In [ ]:
#Feature creation, rankings
#Difference between daily rankings
dates = stategrp['Date'].unique().tolist()
ustemp = pd.DataFrame()
for state in stategrp['Province_State'].unique():
    statetemp = stategrp.loc[stategrp['Province_State']==state].copy().reset_index(drop=True)
    yest = pd.Series(statetemp.loc[statetemp['Date'].isin(dates[1:]),'Total Cases Daily Ranking'].reset_index(drop=True)).rename('Total Cases Ranking Daily Change')
    statetemp = pd.concat([statetemp,yest], axis=1)
    ustemp = pd.concat([ustemp, statetemp])